In [11]:
import csv
import random
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker
fake = Faker()

# Helper function to write data to CSV with '|' delimiter
def write_csv(filename, header, rows):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file, delimiter='|')
        writer.writerow(header)
        writer.writerows(rows)

In [12]:
# 1. DimCustomer: 8 customers
customers = []
for i in range(1, 9):
    customers.append([
        i,  # customer_id
        fake.first_name(),
        fake.last_name(),
        fake.email(),
        fake.address().replace('\n', ', '),
        fake.city(),
        fake.state(),
        fake.postcode(),
        fake.phone_number()
    ])
write_csv('./data/dim_customers.csv', [
    'customer_id', 'first_name', 'last_name', 'email', 'address',
    'city', 'state', 'postal_code', 'phone_number'
], customers)

In [13]:
# 2. DimAccount: 12 accounts
accounts = []
account_id = 1
for cust_id in range(1, 9):
    num_accounts = 2 if cust_id <= 4 else 1  # Customers 1-4 have 2 accounts, 5-8 have 1
    for _ in range(num_accounts):
        accounts.append([
            account_id,
            fake.bban(),  # account_number
            cust_id,  # customer_id
            random.choice(['Checking', 'Savings']),
            round(random.uniform(100, 10000), 2),  # account_balance
            random.randint(300, 850)  # credit_score
        ])
        account_id += 1
write_csv('./data/dim_accounts.csv', [
    'account_id', 'account_number', 'customer_id', 'account_type',
    'account_balance', 'credit_score'
], accounts)

In [15]:
# 3. DimDate: 10 dates from 2023-01-01 to 2023-01-10
dates = []
start_date = datetime(2023, 1, 1)
for i in range(10):
    date = start_date + timedelta(days=i)
    dates.append([
        i + 1,  # date_id
        date.strftime('%Y-%m-%d'),
        date.day,
        date.month,
        date.year,
        (date.month - 1) // 3 + 1,  # quarter (1 for January)
        date.strftime('%A')  # weekday
    ])
write_csv('./data/dim_dates.csv', [
    'date_id', 'date', 'day', 'month', 'year', 'quarter', 'weekday'
], dates)



In [16]:
# 4. DimChannel: 3 channels
channels = [
    [1, 'Online'],
    [2, 'In-Branch'],
    [3, 'ATM']
]
write_csv('./data/dim_channels.csv', 
          ['channel_id', 'channel_name'], channels)

In [17]:
# 5. DimLoan: 5 loans
loans = []
for i in range(1, 6):
    loans.append([
        i,
        random.choice(['Mortgage', 'Auto', 'Personal', 'Student', 'Business']),
        round(random.uniform(1000, 100000), 2),  # loan_amount
        round(random.uniform(1.0, 10.0), 2)  # interest_rate
    ])
write_csv('./data/dim_loans.csv', [
    'loan_id', 'loan_type', 'loan_amount', 'interest_rate'
], loans)

In [18]:
# 6. DimTransactionType: 4 transaction types
txn_types = [
    [1, 'Deposit'],
    [2, 'Withdrawal'],
    [3, 'Transfer'],
    [4, 'Payment']
]
write_csv('./data/dim_transaction_types.csv', [
    'transaction_type_id', 'description'
], txn_types)

In [19]:
# 7. DimLocation: 3 locations
locations = [
    [1, '123 Main St', 'New York', 'NY', 'USA'],
    [2, '456 Oak Ave', 'Los Angeles', 'CA', 'USA'],
    [3, '789 Pine Rd', 'Chicago', 'IL', 'USA']
]
write_csv('./data/dim_locations.csv', [
    'location_id', 'address', 'city', 'state', 'country'
], locations)

In [20]:
# 8. DimCurrency: 3 currencies
currencies = [
    [1, 'USD', 'USD', True],
    [2, 'EUR', 'EUR', True],
    [3, 'GBP', 'GBP', True]
]
write_csv('./data/dim_currencies.csv', [
    'currency_id', 'name', 'iso3_code', 'active'
], currencies)

In [21]:
# 9. DimInvestmentType: 3 investment types
inv_types = [
    [1, 'Stocks'],
    [2, 'Bonds'],
    [3, 'Mutual Funds']
]
write_csv('./data/dim_investment_types.csv', [
    'investment_type_id', 'investment_type_name'
], inv_types)

In [22]:
# 10. FactTransactions: 30 transactions
transactions = []
for i in range(1, 31):
    customer_id = random.randint(1, 8)
    # Select an account_id that belongs to this customer
    account_ids = [acc[0] for acc in accounts if acc[2] == customer_id]
    account_id = random.choice(account_ids)
    date_id = random.randint(1, 10)
    channel_id = random.randint(1, 3)
    txn_type_id = random.randint(1, 4)
    location_id = random.randint(1, 3)
    currency_id = random.randint(1, 3)
    # Assign loan_id or investment_type_id for some transactions
    if i <= 10:
        loan_id = random.randint(1, 5)
        investment_type_id = None
    elif i <= 20:
        loan_id = None
        investment_type_id = random.randint(1, 3)
    else:
        loan_id = None
        investment_type_id = None
    txn_amount = round(random.uniform(10, 1000), 2)
    txn_status = 'Completed'
    transactions.append([
        i, customer_id, date_id, channel_id, account_id, txn_type_id,
        location_id, currency_id, loan_id, investment_type_id, txn_amount, txn_status
    ])
write_csv('./data/fact_transactions.csv', [
    'transaction_id', 'customer_id', 'date_id', 'channel_id', 'account_id',
    'txn_type_id', 'location_id', 'currency_id', 'loan_id', 'investment_type_id',
    'txn_amount', 'txn_status'
], transactions)

In [23]:
# 11. FactDailyBalances: 20 daily balances
daily_balances = []
used_pairs = set()
while len(daily_balances) < 20:
    account_id = random.randint(1, 12)
    date_id = random.randint(1, 10)
    pair = (account_id, date_id)
    if pair not in used_pairs:  # Ensure unique account-date pairs
        used_pairs.add(pair)
        balance = round(random.uniform(100, 10000), 2)
        daily_balances.append([
            len(daily_balances) + 1, date_id, account_id, balance, balance, balance
        ])
write_csv('./data/fact_daily_balances.csv', [
    'balance_id', 'date_id', 'account_id', 'opening_balance',
    'closing_balance', 'average_balance'
], daily_balances)

In [24]:
# 12. FactCustomerInteractions: 15 interactions
interactions = []
for cust_id in range(1, 9):
    num_interactions = random.choice([1, 2])
    dates_used = set()
    for _ in range(num_interactions):
        date_id = random.randint(1, 10)
        if date_id not in dates_used:  # Avoid duplicate dates for same customer
            dates_used.add(date_id)
            interactions.append([
                len(interactions) + 1, date_id, cust_id, random.randint(1, 3),
                random.choice(['Call', 'Email', 'Chat']), random.randint(1, 5)
            ])
write_csv('./data/fact_customer_interactions.csv', [
    'interaction_id', 'date_id', 'customer_id', 'channel_id',
    'interaction_type', 'interaction_rating'
], interactions[:15])  # Limit to 15

In [25]:
# 13. FactLoanPayments: 10 payments (2 per loan)
loan_payments = []
for loan_id in range(1, 6):
    customer_id = loan_id  # Assign loan_id 1 to customer 1, etc., for simplicity
    for _ in range(2):
        date_id = random.randint(1, 10)
        payment_amount = round(random.uniform(100, 1000), 2)
        payment_status = 'Paid'
        loan_payments.append([
            len(loan_payments) + 1, date_id, loan_id, customer_id,
            payment_amount, payment_status
        ])
write_csv('./data/fact_loan_payments.csv', [
    'payment_id', 'date_id', 'loan_id', 'customer_id',
    'payment_amount', 'payment_status'
], loan_payments)

In [27]:
# 14. FactInvestments: 8 investments
investments = []
for inv_type in [[1, 3], [2, 3], [3, 2]]:  # 3 for type 1, 3 for type 2, 2 for type 3
    inv_type_id, count = inv_type
    for _ in range(count):
        investments.append([
            len(investments) + 1, random.randint(1, 10), inv_type_id,
            random.randint(1, 12), round(random.uniform(1000, 10000), 2),
            round(random.uniform(20, 900), 2),
        ])
write_csv('./data/fact_investments.csv', [
    'investment_id', 'date_id', 'investment_type_id', 'account_id', 'amount_invested', 'investment_return'
], investments)